In [1]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.plotting import simple_plot

import pandas as pd
import numpy as np
import cvxpy as cp
import networkx as nx

## Data processing

In [2]:
net = pn.case6ww()

In [3]:
# currently the sn_mva (base power) is inconsistent with that in the original MATPOWER cases
net.sn_mva = 100

In [4]:
n = len(net.bus)

#### Generators

In [5]:
gen_df_list = []
gen_name_list = ["gen", "sgen", "ext_grid"]
data_col_list = ["bus", "max_p_mw", "min_p_mw", "max_q_mvar", "min_q_mvar"]

for gen_name in gen_name_list:
    if not net[gen_name].empty:
        # get a table of cost coefficients only for the current type of generators
        gen_name_poly_cost = net.poly_cost.loc[net.poly_cost.et == gen_name].set_index("element")
        # get a table of cost coefficients and power bounds only for the current type of generators
        gen_name_df = net[gen_name][data_col_list].join(gen_name_poly_cost)
        gen_df_list.append(gen_name_df)

# combine tables for all types of generators
gen_df = pd.concat(gen_df_list)
n_gen = len(gen_df)

In [6]:
gen_df

,bus,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,1,150.0,37.5,100.0,-100.0,gen,200.0,10.333,0.00889,0.0,0.0,0.0
1,2,180.0,45.0,100.0,-100.0,gen,240.0,10.833,0.00741,0.0,0.0,0.0
0,0,200.0,50.0,100.0,-100.0,ext_grid,213.1,11.669,0.00533,0.0,0.0,0.0


#### Bus loads

In [7]:
load_df = net.bus.join(net.load[["bus", "p_mw", "q_mvar"]].set_index("bus")).fillna(0)[["p_mw", "q_mvar"]]

In [8]:
load_df

,p_mw,q_mvar
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,70.0,70.0
4,70.0,70.0
5,70.0,70.0


#### Admittance matrices

In [9]:
# obtain a NetworkX Graph from the network, with each edge containing p.u. impedance data
graph = pp.topology.create_nxgraph(net, multi=False, calc_branch_impedances=True, branch_impedance_unit="pu")

In [10]:
G_mat = np.zeros((n,n))
B_mat = np.zeros((n,n))
for i,j in graph.edges:
    edge = graph.edges[(i,j)]
    r = edge["r_pu"]
    x = edge["x_pu"]
    G_mat[i][j] = 1/r
    G_mat[j][i] = 1/r
    B_mat[i][j] = 1/x
    B_mat[j][i] = 1/x

In [11]:
for _, row in net.shunt.iterrows():
    i = row["bus"]
    G_mat[i][i] = row["p_mw"]
    B_mat[i][i] = -row["q_mvar"] 

## Variables

In [12]:
# X = VV*
X = cp.Variable((n,n), hermitian=True)
# active power generated
p_g = cp.Variable((n_gen, 1))
# reactive power generated
q_g = cp.Variable((n_gen, 1))

## Parameters

In [13]:
# loads
p_d = cp.Parameter((n,1), nonneg=True, value=load_df[["p_mw"]].to_numpy())
q_d = cp.Parameter((n,1), nonneg=True, value=load_df[["q_mvar"]].to_numpy())

# admittance matrices
G = cp.Parameter((n,n), nonneg=True, value=G_mat)
B = cp.Parameter((n,n), nonneg=True, value=B_mat)

# squares of voltage bounds
V_min_sq = cp.Parameter((n), nonneg=True, value=np.square(net.bus["min_vm_pu"].to_numpy()))
V_max_sq = cp.Parameter((n), nonneg=True, value=np.square(net.bus["max_vm_pu"].to_numpy()))

# bounds for generated power
p_min = cp.Parameter((n_gen,1), value=gen_df[["min_p_mw"]].to_numpy())
p_max = cp.Parameter((n_gen,1), value=gen_df[["max_p_mw"]].to_numpy())
q_min = cp.Parameter((n_gen,1), value=gen_df[["min_q_mvar"]].to_numpy())
q_max = cp.Parameter((n_gen,1), value=gen_df[["max_q_mvar"]].to_numpy())

## Constraints

In [14]:
constraints = [X >> 0]

In [35]:
gen_df

,bus,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,1,150.0,37.5,100.0,-100.0,gen,200.0,10.333,0.00889,0.0,0.0,0.0
1,2,180.0,45.0,100.0,-100.0,gen,240.0,10.833,0.00741,0.0,0.0,0.0
0,0,200.0,50.0,100.0,-100.0,ext_grid,213.1,11.669,0.00533,0.0,0.0,0.0


In [33]:
load_df

,p_mw,q_mvar
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,70.0,70.0
4,70.0,70.0
5,70.0,70.0


In [15]:
for i in range(n):
    gen_list = gen_df.loc[gen_df["bus"] == i].index.to_numpy()
    constraints += [
        cp.sum([p_g[k] for k in gen_list]) - p_d[i] == G[i][i] * X[i][i] +
        cp.sum([G[i][j] * cp.real(X[i][j]) + B[i][j] * cp.imag(X[i][j]) for j in graph.neighbors(i)])
    ]
    constraints += [
        cp.sum([q_g[k] for k in gen_list]) - q_d[i] == -B[i][i] * X[i][i] +
        cp.sum([-B[i][j] * cp.real(X[i][j]) + G[i][j] * cp.imag(X[i][j]) for j in graph.neighbors(i)])
    ]

In [16]:
constraints += [
    cp.real(cp.diag(X)) >= V_min_sq,
    cp.real(cp.diag(X)) <= V_max_sq
]

In [17]:
constraints += [
    p_g >= p_min,
    p_g <= p_max,
    q_g >= q_min,
    q_g <= q_max
]

## Solving

In [18]:
p_cost = cp.sum(gen_df["cp2_eur_per_mw2"].to_numpy() @ cp.square(p_g) +
                gen_df["cp1_eur_per_mw"].to_numpy() @ p_g +
                gen_df[["cp0_eur"]].to_numpy())
q_cost = cp.sum(gen_df["cq2_eur_per_mvar2"].to_numpy() @ cp.square(q_g) +
                gen_df["cq2_eur_per_mvar2"].to_numpy() @ q_g +
                gen_df[["cp0_eur"]].to_numpy())

In [19]:
prob = cp.Problem(cp.Minimize(p_cost + q_cost), constraints)

In [20]:
prob.is_dcp()

True

In [21]:
prob.is_dpp()

True

In [22]:
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is inf
A solution X is
None


In [23]:
prob.status

'infeasible'

In [24]:
print(p_g.value)

None
